# Final Project

## Predict whether a mammogram mass is benign or malignant

We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Your assignment

Build a Multi-Layer Perceptron and train it to classify masses as benign or malignant based on its features.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.

Remember to normalize your data first! And experiment with different topologies, optimizers, and hyperparameters.

I was able to achieve over 80% accuracy - can you beat that?


## Let's begin: prepare your data

Start by importing the mammographic_masses.data.txt file into a Pandas dataframe (hint: use read_csv) and take a look at it.

In [3]:
import pandas as pd
import numpy as np

features = ['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity']
data = pd.read_csv('mammographic_masses.data.txt', names=features, na_values='?')
data.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


Make sure you use the optional parmaters in read_csv to convert missing data (indicated by a ?) into NaN, and to add the appropriate column names (BI_RADS, age, shape, margin, density, and severity):

Evaluate whether the data needs cleaning; your model is only as good as the data it's given. Hint: use describe() on the dataframe.

In [4]:
data.describe()
#max number: 961
#column with most missing values: Density

,BI-RADS,Age,Shape,Margin,Density,Severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


There are quite a few missing values in the data set. Before we just drop every row that's missing data, let's make sure we don't bias our data in doing so. Does there appear to be any sort of correlation to what sort of data has missing fields? If there were, we'd have to try and go back and fill that data in.

In [5]:
data.corr()

,BI-RADS,Age,Shape,Margin,Density,Severity
BI-RADS,1.000000,0.094754,0.186055,0.163086,0.038729,0.231601
Age,0.094754,1.000000,0.364077,0.411355,0.028954,0.432066
Shape,0.186055,0.364077,1.000000,0.742211,0.078666,0.563308
Margin,0.163086,0.411355,0.742211,1.000000,0.109392,0.574919
Density,0.038729,0.028954,0.078666,0.109392,1.000000,0.064010
Severity,0.231601,0.432066,0.563308,0.574919,0.064010,1.000000


If the missing data seems randomly distributed, go ahead and drop rows with missing data. Hint: use dropna().

In [8]:
data.dropna(inplace=True)
data.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


Next you'll need to convert the Pandas dataframes into numpy arrays that can be used by scikit_learn. Create an array that extracts only the feature data we want to work with (age, shape, margin, and density) and another array that contains the classes (severity). You'll also need an array of the feature name labels.

In [9]:
features = np.array(data[['Age', 'Shape', 'Margin', 'Density']])
featurenames = ['Age', 'Shape', 'Margin', 'Density']
label = np.array(data['Severity'])

Some of our models require the input data to be normalized, so go ahead and normalize the attribute data. Hint: use preprocessing.StandardScaler().

In [10]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
features = scale.fit_transform(features)
features[:5]

array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       [ 0.08307148, -1.43517241,  1.39618483,  0.24046607],
       [ 1.37885518, -1.43517241,  0.75770912,  0.24046607]])

## Now build your neural network.

Now set up an actual MLP model using Keras:

In [51]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

def create_model():
    model = Sequential()
    model.add(Dense(6, activation='relu', kernel_initializer='normal', input_dim=4))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return model

#actually runs model
history = model.fit(features, label, batch_size=6, epochs=10, verbose=1)
    

Train on 830 samples
Epoch 1/10
830/830 [==============================] - 0s 517us/sample - loss: 0.4560 - accuracy: 0.8120
Epoch 2/10
830/830 [==============================] - 0s 472us/sample - loss: 0.4526 - accuracy: 0.8145
Epoch 3/10
830/830 [==============================] - 0s 568us/sample - loss: 0.4543 - accuracy: 0.8108
Epoch 4/10
830/830 [==============================] - 0s 554us/sample - loss: 0.4569 - accuracy: 0.8072
Epoch 5/10
830/830 [==============================] - 0s 550us/sample - loss: 0.4649 - accuracy: 0.8145
Epoch 6/10
830/830 [==============================] - 1s 627us/sample - loss: 0.4541 - accuracy: 0.8108
Epoch 7/10
830/830 [==============================] - 1s 615us/sample - loss: 0.4559 - accuracy: 0.8084
Epoch 8/10
830/830 [==============================] - 0s 593us/sample - loss: 0.4560 - accuracy: 0.8108
Epoch 9/10
830/830 [==============================] - 0s 580us/sample - loss: 0.4611 - accuracy: 0.8096
Epoch 10/10
830/830 [======================

In [23]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 830, 830)          4150      
_________________________________________________________________
dropout_6 (Dropout)          (None, 830, 830)          0         
_________________________________________________________________
dense_12 (Dense)             (None, 830, 60)           49860     
_________________________________________________________________
dropout_7 (Dropout)          (None, 830, 60)           0         
_________________________________________________________________
dense_13 (Dense)             (None, 830, 1)            61        
Total params: 54,071
Trainable params: 54,071
Non-trainable params: 0
_________________________________________________________________


In [53]:
#CV scores
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
cv_scores = cross_val_score(estimator, features, label, cv=10)
cv_scores.mean()

0.7987951815128327

## How did you do?

Which topology, and which choice of hyperparameters, performed the best? Feel free to share your results!